# 드라이브 연결

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import random
import glob
import re

import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from tqdm import tqdm

# Fixed RandomSeed & Setting Hyperparameter _ 필요한가 고민

In [ ]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
LOOKBACK, PREDICT, BATCH_SIZE, EPOCHS = 28, 7, 16, 50
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# train 불러오기

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/lgaimers7기/data/train/train.csv')

# Define Model_ 고민

In [ ]:
class MultiOutputLSTM(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=64, num_layers=2, output_dim=7):
        super(MultiOutputLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])  # (B, output_dim)

# Feature Engineeering

[원시 데이터]  

  ↓

[피처 만들기]
  - 날짜 특징 (요일, 월, 주차, 주말 여부 등)
  - 시계열 특징 (최근 7일 평균, 14일 평균, 이전 매출량 등)
  - 업장·메뉴 ID 인코딩
  - 시즌/성수기 플래그

  ↓

[모델 학습]

  - Horizon=1~7 각각 별도 회귀 모델 (LightGBM)
  - 업장 가중치 적용 (담하, 미라시아 비중↑)

  ↓

[예측]
  - 예측 결과 7일치 병합

  ↓

[제출 포맷 변환]

# 한번에 돌아가는 코드

1 데이터 로드 (train, test)

2 전처리 & 피처 생성

- 날짜 기반 특징

- Lag & Rolling

- 카테고리 인코딩

3 LightGBM 학습

- day+1 ~ day+7 각각 별도 모델

- 업장 가중치 적용

4 예측

5 제출 포맷 변환


그럼 이번엔 제가 아까 준 LightGBM 전역 모델 코드에다가

공휴일 / 성수기 피처

Seasonal Naïve + 이동평균 앙상블

Optuna를 이용한 하이퍼파라미터 튜닝

까지 풀패키지로 한 번에 넣어서 드릴게요.
이렇게 하면 LSTM 베이스라인보다 리더보드에서 안정적으로 점수 향상을 기대할 수 있습니다.

시드 고정안 하고 그냥 돌림

In [4]:
!pip  install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 24.6 MB/s eta 0:00:00


In [8]:
import os
import glob
import re
import pandas as pd
import numpy as np
import lightgbm as lgb
import holidays
import optuna
from tqdm import tqdm

# -----------------------
# 1. 데이터 로드
# -----------------------
train = pd.read_csv('/content/drive/MyDrive/lgaimers7기/data/train/train.csv')
sample_submission = pd.read_csv('/content/drive/MyDrive/lgaimers7기/data/sample_submission.csv')

test_files = sorted(glob.glob('/content/drive/MyDrive/lgaimers7기/data/test/TEST_*.csv'))
test_list = []
for path in test_files:
    df = pd.read_csv(path)
    df['test_prefix'] = re.search(r'(TEST_\d+)', os.path.basename(path)).group(1)
    test_list.append(df)
test_all = pd.concat(test_list, ignore_index=True)


if '영업장명' not in train.columns:
    train[['영업장명','메뉴명']] = train['영업장명_메뉴명'].str.split('_', n=1, expand=True)

if '영업장명' not in test_all.columns:
    test_all[['영업장명','메뉴명']] = test_all['영업장명_메뉴명'].str.split('_', n=1, expand=True)





# -----------------------
# 2. 피처 생성 함수
# -----------------------
def create_features(df):
    df['영업일자'] = pd.to_datetime(df['영업일자'])
    df['year'] = df['영업일자'].dt.year
    df['month'] = df['영업일자'].dt.month
    df['day'] = df['영업일자'].dt.day
    df['dayofweek'] = df['영업일자'].dt.dayofweek
    df['weekofyear'] = df['영업일자'].dt.isocalendar().week.astype(int)
    df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)

    # 공휴일 여부
    kr_holidays = holidays.KR(years=df['year'].unique())
    df['is_holiday'] = df['영업일자'].isin(kr_holidays).astype(int)

    # 성수기 (7~8월, 12~1월)
    df['is_peak'] = df['month'].isin([7,8,12,1]).astype(int)
    return df

train = create_features(train)

# Lag & Rolling
def add_lag_features(df, group_cols, target_col, lags, roll_windows):
    df = df.sort_values(['영업장명', '메뉴명', '영업일자'])
    for lag in lags:
        df[f'lag_{lag}'] = df.groupby(group_cols)[target_col].shift(lag)
    for w in roll_windows:
        df[f'roll_mean_{w}'] = df.groupby(group_cols)[target_col].shift(1).rolling(w).mean()
        df[f'roll_std_{w}'] = df.groupby(group_cols)[target_col].shift(1).rolling(w).std()
    return df

train = add_lag_features(train, ['영업장명', '메뉴명'], '매출수량', lags=[1,2,3,7,14], roll_windows=[7,14])


# 문자열 -> category 변환
cat_cols = ['영업장명', '메뉴명', '영업장명_메뉴명']
for col in cat_cols:
    if col in train.columns:
        train[col] = train[col].astype('category')
    if col in test_all.columns:
        test_all[col] = test_all[col].astype('category')


# 업장 가중치
store_weights = train['영업장명'].map(lambda x: 3.0 if x in ['담하','미라시아'] else 1.0)

# -----------------------
# 3. Optuna 튜닝 (Horizon=1만)
# -----------------------
PRED_DAYS = 7
features = [col for col in train.columns if col not in ['영업일자','매출수량'] + [f'y_{i}' for i in range(1,PRED_DAYS+1)]]

train[f'y_1'] = train.groupby(['영업장명','메뉴명'])['매출수량'].shift(-1)
df_train = train.dropna(subset=['y_1'])
X_train = df_train[features]
y_train = df_train['y_1']

def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'mae',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 15, 63),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.7, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.7, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 5.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 0, 5.0),
        'seed': 42
    }
    lgb_train = lgb.Dataset(X_train, y_train, weight=store_weights.loc[df_train.index])
    model = lgb.train(params, lgb_train, num_boost_round=300)
    preds = model.predict(X_train)
    return np.mean(np.abs(y_train - preds))

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)
best_params = study.best_params
best_params.update({'objective':'regression','metric':'mae','seed':42})

# -----------------------
# 4. Multi-Horizon 모델 학습
# -----------------------
models = {}
for day_ahead in range(1, PRED_DAYS+1):
    train[f'y_{day_ahead}'] = train.groupby(['영업장명','메뉴명'])['매출수량'].shift(-day_ahead)
    df_train = train.dropna(subset=[f'y_{day_ahead}'])
    X = df_train[features]
    y = df_train[f'y_{day_ahead}']
    lgb_train = lgb.Dataset(X, y, weight=store_weights.loc[df_train.index])
    model = lgb.train(best_params, lgb_train, num_boost_round=500)
    models[day_ahead] = model

# -----------------------
# 5. 테스트 데이터 준비
# -----------------------
test_all = create_features(test_all)
for col in ['영업장명', '메뉴명']:
    test_all[col] = test_all[col].astype('category')

full_data = pd.concat([train, test_all], sort=False)
full_data = add_lag_features(full_data, ['영업장명', '메뉴명'], '매출수량', lags=[1,2,3,7,14], roll_windows=[7,14])

# -----------------------
# 6. 예측 + 앙상블
# -----------------------
pred_results = []
for test_prefix, group in full_data[full_data['test_prefix'].notnull()].groupby('test_prefix'):
    group = group.sort_values('영업일자')
    for day_ahead in range(1, PRED_DAYS+1):
        X_test = group[features]
        pred_lgb = models[day_ahead].predict(X_test)

        # Seasonal Naive: lag_7
        pred_sn = group['lag_7'].values

        # Moving Average(7일)
        pred_ma = group['roll_mean_7'].values

        # 앙상블
        preds = 0.6*pred_lgb + 0.2*pred_sn + 0.2*pred_ma
        group[f'pred_{day_ahead}'] = np.clip(preds, 0, None)
    pred_results.append(group)

pred_df = pd.concat(pred_results)

# -----------------------
# 7. 제출 변환 (수정)
# -----------------------
submit_dict = {}
for _, row in pred_df.iterrows():
    # 업장명_메뉴명 문자열로 합치기
    menu_str = f"{row['영업장명']}_{row['메뉴명']}"
    for day_ahead in range(1, PRED_DAYS+1):
        key = (f"{row['test_prefix']}+{day_ahead}일", menu_str)
        submit_dict[key] = row[f'pred_{day_ahead}']

final_df = sample_submission.copy()
for idx in final_df.index:
    date = final_df.loc[idx, '영업일자']
    for col in final_df.columns[1:]:
        final_df.loc[idx, col] = submit_dict.get((date, col), 0)

final_df.to_csv('lgbm_optuna_ensemble_submission.csv', index=False, encoding='utf-8-sig')
print("제출 파일 저장 완료: lgbm_optuna_ensemble_submission.csv")

/tmp/ipython-input-757050905.py:50: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df['is_holiday'] = df['영업일자'].isin(kr_holidays).astype(int)
/tmp/ipython-input-757050905.py:89: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  train[f'y_1'] = train.groupby(['영업장명','메뉴명'])['매출수량'].shift(-1)
[I 2025-08-13 21:57:46,965] A new study created in memory with name: no-name-fb40d1ce-fc7c-4c1a-a55e-ee2ed95d252f
[I 2025-08-13 21:57:51,399] Trial 0 finished with value: 6.190964198917949 and parameters: {'learning_rate': 0.030338253714452673, 'num_leaves': 15, 'feature_fraction': 0.7247908519235143, 'bagging_

제출 파일 저장 완료: lgbm_optuna_ensemble_submission.csv


좋아요, 그럼 지금까지 당신 코드에 적용한 모든 변경 사항을 단계별로 정리해 드릴게요.
원본은 LSTM 베이스라인이었는데, 지금은 **전역 LightGBM + Optuna 튜닝 + 피처 엔지니어링 + 앙상블 + 제출 포맷 수정** 구조로 완전히 변했습니다.

---

## 1. **모델 구조 변경**

* **기존**: 업장×메뉴별 개별 LSTM, 시계열만 입력, horizon=7 멀티 아웃풋
* **변경**: 전 업장·메뉴 데이터를 합쳐서 전역(Global) LightGBM 모델로 학습

  * Horizon=1\~7 각각 별도 모델 → Direct Multi-Horizon 방식
  * `담하`, `미라시아` 업장은 샘플 가중치 3배 적용 (리더보드 유리)

---

## 2. **데이터 로드 & 컬럼 분리**

* `train.csv`, `sample_submission.csv`, `TEST_*.csv` 모두 로드
* **LSTM 포맷 호환을 위해**:

  ```python
  if '영업장명' not in train.columns:
      train[['영업장명','메뉴명']] = train['영업장명_메뉴명'].str.split('_', n=1, expand=True)
  if '영업장명' not in test_all.columns:
      test_all[['영업장명','메뉴명']] = test_all['영업장명_메뉴명'].str.split('_', n=1, expand=True)
  ```

  → 업장명/메뉴명 분리해 groupby와 가중치 계산 가능하게 함.

---

## 3. **피처 엔지니어링**

### (1) 날짜 기반 특징 (`create_features`)

* `year`, `month`, `day`, `dayofweek`, `weekofyear`, `is_weekend`
* **공휴일 플래그**: `holidays.KR` 라이브러리로 연도별 공휴일 생성
* **성수기 플래그**: 7,8,12,1월 → 1

### (2) Lag & Rolling (`add_lag_features`)

* Lag: 1, 2, 3, 7, 14일 전 매출
* Rolling Mean & Std: 최근 7, 14일 평균·표준편차

### (3) 카테고리 인코딩

* LightGBM 입력 시 `영업장명`, `메뉴명`, `영업장명_메뉴명`을 `.astype('category')`

---

## 4. **Optuna 하이퍼파라미터 튜닝**

* Horizon=1 모델만 대상으로 Optuna 실행 (30 trials)
* 탐색 파라미터:

  * `learning_rate`
  * `num_leaves`
  * `feature_fraction`
  * `bagging_fraction`
  * `bagging_freq`
  * `lambda_l1`
  * `lambda_l2`
* 최적 파라미터를 나머지 Horizon에도 동일 적용

---

## 5. **Multi-Horizon 학습**

* day+1 \~ day+7 각각 별도의 LightGBM 회귀 모델 학습
* 각 모델에 업장 가중치 반영

---

## 6. **예측 및 앙상블**

* LightGBM 예측 + Seasonal Naive(lag\_7) + 이동평균(roll\_mean\_7)
* 가중치:

  * LightGBM: 0.6
  * Seasonal Naive: 0.2
  * Moving Average: 0.2
* `np.clip(..., 0, None)`로 음수 예측값 0으로 보정

---

## 7. **제출 포맷 변환 (0 나오는 문제 해결)**

* **기존 문제**: `submit_dict` 키가 `(업장명, 메뉴명)` tuple인데, `sample_submission`의 컬럼은 `'업장명_메뉴명'` 문자열 → 매칭 실패 → 전부 0
* **수정**:

  ```python
  menu_str = f"{row['영업장명']}_{row['메뉴명']}"
  key = (f"{row['test_prefix']}+{day_ahead}일", menu_str)
  ```

  → `submit_dict` 키와 `sample_submission` 컬럼명을 동일한 문자열 형식으로 통일
* 이렇게 해서 예측값이 정상적으로 들어가도록 수정

---

## 8. **기타 개선**

* `FutureWarning`는 남아 있지만 동작에는 영향 없음 (원하면 dtype 명시로 제거 가능)
* 전체 파이프라인이 **train → Optuna → 7모델 학습 → test 예측 → 제출 변환**까지 한 번에 실행되도록 구성

---

정리하면, 지금 돌리고 있는 건:

> **전역 LightGBM + 풍부한 피처 + 가중치 적용 + Optuna 튜닝 + 앙상블 + 제출 포맷 수정**
> LSTM 대비 데이터 효율성과 예측 안정성이 크게 향상된 구조

---

원하면 제가 여기서 **FutureWarning 제거**까지 해서 깔끔한 최종본 코드로 만들어 드릴 수 있어요.
그렇게 하면 경고 없이 실행되고, 결과도 동일하게 나옵니다.
그걸 해드릴까요?


## EDA